# Players Load
This will load the points for every player over every week. The only way to do this is to ask for each week separately. This may take a few minutes to run. This will allow us to test how well a team was managed based on how the players played.

Creates `players_y.csv` with `y` being the year.

In [27]:
import os
import requests
import pandas as pd
import json

from dotenv import load_dotenv

from functools import reduce, partial

load_dotenv()
league_id = os.getenv('league_id')
current_year = int(os.getenv('current_year'))
url = partial("https://fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{league_id}".format, league_id=league_id)
history_url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}".format(league_id)
cookies = {
        "swid": os.getenv('swid'),
        "espn_s2": os.getenv('espn_s2')
    }


In [28]:
teams = pd.read_csv("teams.csv", usecols=['id', 'year'])
cur_teams = teams.loc[teams["year"] == current_year]
hist_teams = teams.loc[teams["year"].isin([2019, 2020])]

In [20]:
def flatmap(func, iter):
    return reduce(list.__add__, map(func, iter))

def unwrap(req):
    return req.json()[0]

def pass_through(req):
    return req.json()


In [22]:
def current_year_game(roster_key, scoringPeriodId, seasonId, load_url, func):
    req = requests.get(load_url,
    cookies=cookies,
    params={"view": ["mRoster", "mMatchup", "mMatchupScore"], "scoringPeriodId": scoringPeriodId, "seasonId": seasonId})
    schedule = func(req)["schedule"]
    find_roster = list(filter(lambda x: roster_key in x["away"] or roster_key in x["home"], schedule))
    return find_roster

def get_game_data(roster_key, game):
    away_team_id = game["away"]["teamId"]
    scoringPeriodId = game["matchupPeriodId"]
    away_roster = list(map(lambda x: get_player_game_data(x, scoringPeriodId, away_team_id), game["away"][roster_key]["entries"]))
    home_team_id = game["home"]["teamId"]
    home_roster = list(map(lambda x: get_player_game_data(x, scoringPeriodId, home_team_id), game["home"][roster_key]["entries"]))
    return away_roster + home_roster

def get_player_game_data(player, scoringPeriodId, teamId):
    name = player['playerPoolEntry']['player']['fullName']
    slotId = player['lineupSlotId']
    injury = player['playerPoolEntry']['player']['injuryStatus'] if 'injuryStatus' in player['playerPoolEntry']['player'] else 'NA'
    year = max([stat['seasonId'] for stat in player['playerPoolEntry']['player']['stats']] + [0])
    actual = ([stat['appliedTotal'] for stat in player['playerPoolEntry']['player']['stats'] if stat['scoringPeriodId'] == scoringPeriodId and stat['statSourceId'] == 0] + [0.0])[0]
    projected = ([stat['appliedTotal'] for stat in player['playerPoolEntry']['player']['stats'] if stat['scoringPeriodId'] == scoringPeriodId and stat['statSourceId'] == 1] + [0.0])[0]
    slots = player['playerPoolEntry']['player']['eligibleSlots']
    playerId = player['playerId']
    return [name, slotId, scoringPeriodId, year, teamId, injury, actual, projected, slots, playerId]

cur_game = partial(current_year_game, "rosterForCurrentScoringPeriod")
hist_game = partial(current_year_game, "rosterForMatchupPeriod")

cur_game_data = partial(get_game_data, "rosterForCurrentScoringPeriod")
hist_game_data = partial(get_game_data, "rosterForMatchupPeriod")

In [23]:
def get_all_periods(teamId, year):
    return list(map(lambda x: (x, teamId, year), range(1,14)))

def get_game_for_team(teamId, scoringPeriodId, seasonId, load_url, func):
    req = requests.get(load_url,
    cookies=cookies,
    params={"view": "mRoster", "scoringPeriodId": scoringPeriodId, "seasonId": seasonId, "forTeamId": teamId})
    roster = func(req)["teams"][0]["roster"]["entries"]
    return roster

def get_player_data(player):
    name = player['playerPoolEntry']['player']['fullName']
    slotId = player['lineupSlotId']
    score = player['playerPoolEntry']['appliedStatTotal']
    slots = player['playerPoolEntry']['player']['eligibleSlots']
    playerId = player['playerId']
    year = max([stat['seasonId'] for stat in player['playerPoolEntry']['player']['stats']] + [0])
    scoringPeriodId = max([stat['scoringPeriodId'] for stat in player['playerPoolEntry']['player']['stats']] + [0])
    status = player['playerPoolEntry']['player']['injuryStatus'] if 'injuryStatus' in player['playerPoolEntry']['player'] else 'NA'
    teamId = player['playerPoolEntry']['onTeamId']
    return [name, slotId, scoringPeriodId, year, teamId, status, score, slots, playerId]
    

In [30]:
games = flatmap(lambda x: get_game_for_team(x[1], x[0], x[2], url(year=x[2]), pass_through) ,flatmap(lambda x: get_all_periods(x[0], x[1]), cur_teams.values))


In [31]:
players = list(map(get_player_data, games))
cur_players_df = pd.DataFrame(players, columns=['name', 'position_id', 'week', 'year', 'team_id', 'status', 'points', 'slots', 'player_id'])

In [33]:
cur_players_df.to_csv("players_{}.csv".format(current_year))

In [34]:
hist_games = flatmap(lambda x: get_game_for_team(x[1], x[0], x[2], history_url, unwrap) ,flatmap(lambda x: get_all_periods(x[0], x[1]), hist_teams.values))

In [35]:
hist_players = list(map(get_player_data, hist_games))
hist_players_df = pd.DataFrame(hist_players, columns=['name', 'position_id', 'week', 'year', 'team_id', 'status', 'points', 'slots', 'player_id'])

In [36]:
hist_players_df.head()

,name,position_id,week,year,team_id,status,points,slots,player_id
0,Michael Thomas,4,1,2019,1,ACTIVE,24.3,"[3, 4, 5, 23, 7, 20, 21]",2976316
1,Melvin Gordon,2,1,2019,1,ACTIVE,0.0,"[2, 3, 23, 7, 20, 21]",2576434
2,Marlon Mack,2,1,2019,1,ACTIVE,34.4,"[2, 3, 23, 7, 20, 21]",3139605
3,Allen Robinson II,4,1,2019,1,ACTIVE,19.2,"[3, 4, 5, 23, 7, 20, 21]",16799
4,49ers D/ST,16,1,2019,1,NA,35.0,"[16, 20, 21]",-16025


In [37]:
for y in hist_players_df["year"].unique():
    hist_players_df.loc[hist_players_df["year"] == y].to_csv("players_{}.csv".format(y))